In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from graphviz import Digraph

In [33]:
# Function to clean column names
def clean_column_name(name):
    # Convert to lowercase
    name = name.lower()
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    # Remove special characters (except underscores)
    name = re.sub(r'[^\w\s]', '', name)
    return name

In [34]:
# Function to process columns and split rows
def process_column(df, column_name):
    new_rows = []
    for index, row in df.iterrows():
         # Ensure the column values are strings
        value = str(row[column_name])
        # Split the values in the column based on spaces and strip any extra spaces
        values = [val.strip() for val in row[column_name].split() if val.strip()]
        
        # If there's more than one value, create a new row for each value
        if len(values) > 1:
            for value in values:
                new_row = row.copy()  # Copy the original row
                new_row[column_name] = value  # Update with the trimmed text
                new_rows.append(new_row)
        else:
            # Keep the original row if there's only one value (no split needed)
            new_rows.append(row)
    return pd.DataFrame(new_rows)

In [35]:
# Function to clean data in a column
def clean_column_data(series):
    # Remove special characters and trim spaces
    series = series.apply(lambda x: re.sub(r'[^\w\s]', '', str(x)).strip() if pd.notna(x) else x)
    return series

In [36]:
def clean_account_no(value):
    if pd.isna(value):
        return value  # Return NaN if the value is NaN
    # Split the account number by space
    parts = value.split()
    # Check if all the parts are the same
    if len(parts) > 1 and all(part == parts[0] for part in parts):
        # If they are the same, return only one instance
        return parts[0]
    # If not, return the original value (no change needed)
    return value

In [37]:
def format_amount_indian(amount):
    # Convert the amount to a string and remove any existing commas
    amount_str = str(amount).replace(',', '')
    
    # Check if the number has more than 3 digits
    if len(amount_str) > 3:
        # Get the last 3 digits
        last_three = amount_str[-3:]
        # Get the remaining digits
        remaining = amount_str[:-3]
        # Group digits in thousands (group of 2 after the first group of 3)
        grouped = [remaining[max(0, i-2):i] for i in range(len(remaining), 0, -2)]
        # Reverse and join the grouped digits with commas
        formatted_remaining = ','.join(grouped[::-1])
        # Concatenate the formatted remaining part with the last three digits
        formatted_amount = f'{formatted_remaining},{last_three}'
    else:
        # For amounts less than or equal to 999, no formatting is needed
        formatted_amount = amount_str

    return formatted_amount

In [38]:
def split_text(text, max_width=30):
    """
    Split the text into lines, so that no line exceeds max_width characters.
    """
    words = text.split(' ')
    lines = []
    current_line = []
    current_length = 0
    
    for word in words:
        if current_length + len(word) + 1 > max_width:  # +1 for space
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = len(word)
        else:
            current_line.append(word)
            current_length += len(word) + 1  # +1 for space
    
    if current_line:
        lines.append(' '.join(current_line))
    
    return '<BR/>'.join(lines)

In [39]:
# Read the CSV file
file_path = 'D:\\test\\CR.NO 75-2024-1.xlsx'  # Replace with your file path
df = pd.read_excel(file_path, dtype= str)
new_columns = [
            'S No.', 'acknowledgement_no', 'transaction_id', 'Layer', 'to_account_no', 
            'Action Taken by Bank/ (Wallet /PG/PA)/ Merchant// Insurance', 
            'Bank/ (Wallet /PG/PA)/ Merchant / Insurance', 'from_account_no', 'Ifsc Code', 
            'Cheque No', 'MID', 'TID', 'Approval Code', 'Merchant Name', 'Transaction Date', 
            'utr_number', 'amount', 'Reference No', 'Remarks', 'Date of Action', 
            'Action Taken By bank', 'Action Taken Name', 'Action Taken By Email', 
            'Branch Location', 'Branch Manager Name & Contact Details'
        ]
        # Get the current number of columns in the Excel file
num_current_columns = len(df.columns)
num_new_columns = len(new_columns)

# Adjust new columns if necessary (truncate or extend)
if num_current_columns > num_new_columns:
    # If more columns in the Excel, extend new_columns with 'Unnamed' columns
    extra_columns = [f"Unnamed {i+1}" for i in range(num_current_columns - num_new_columns)]
    adjusted_columns = new_columns + extra_columns
else:
    # If fewer columns in the Excel, truncate the new_columns to match
    adjusted_columns = new_columns[:num_current_columns]
# Replace the column headers with the adjusted columns
df.columns = adjusted_columns
df.columns = [clean_column_name(col) for col in df.columns]
df.columns

Index(['s_no', 'acknowledgement_no', 'transaction_id', 'layer',
       'to_account_no', 'action_taken_by_bank_wallet_pgpa_merchant_insurance',
       'bank_wallet_pgpa_merchant__insurance', 'from_account_no', 'ifsc_code',
       'cheque_no', 'mid', 'tid', 'approval_code', 'merchant_name',
       'transaction_date', 'utr_number', 'amount', 'reference_no', 'remarks',
       'date_of_action', 'action_taken_by_bank', 'action_taken_name',
       'action_taken_by_email', 'branch_location',
       'branch_manager_name__contact_details', 'unnamed_1', 'unnamed_2'],
      dtype='object')

In [40]:
# Create a list to hold new rows
new_rows = []
# List of columns where you want to append '`'
columns_to_append = ['acknowledgement_no','transaction_id','to_account_no', 'from_account_no','utr_number']

# Prepend '`' to every value in the specified columns
for column in columns_to_append:
    if column in df.columns:
        df[column] = df[column].apply(lambda x: '`' + str(x) if pd.notnull(x) else x)


In [41]:
# Ensure that the 'to_account_no' column is treated as a string
df['from_account_no'] = df['from_account_no'].astype(str)

# Perform the split and handle cases where there's no bracket '[' in the string
df_split = df['from_account_no'].str.split(r'\[', n=1, expand=True)

# Ensure that df_split has two columns by filling missing values with empty strings
df_split[1] = df_split[1].fillna('')  # This handles rows without the '['

# Assign the first part to 'to_account_no' and the second part to 'reported_info'
df['from_account_no'] = df_split[0].str.strip()
df['reported_info'] = df_split[1].str.replace(']', '').str.strip()

# Extract only the number from 'reported_info' (e.g., 'Reported 1 times' -> '1')
df['reported_info'] = df['reported_info'].str.extract(r'(\d+)')

In [42]:
df['reported_info'].unique()

array(['1', nan, '3', '2'], dtype=object)

In [43]:
df.describe

<bound method NDFrame.describe of     s_no acknowledgement_no           transaction_id layer  \
0      1    `30209240022092  `PUNBR52024090612314232     2   
1      2    `30209240022092  `PUNBR52024090612314232     2   
2      3    `30209240022092  `PUNBR52024090612314232     2   
3      4    `30209240022092  `PUNBR52024090612314232     2   
4      5    `30209240022092  `PUNBR52024090612314232     2   
..   ...                ...                      ...   ...   
148  149    `30209240022092  `DBSSR52024090605166760     3   
149  150    `30209240022092  `UJVNN52024090646162643     3   
150  151    `30209240022092  `UJVNN52024090646162610     3   
151  152    `30209240022092          `OtrCyfe9snuywn     4   
152  153    `30209240022092            `425011000249     5   

            to_account_no action_taken_by_bank_wallet_pgpa_merchant_insurance  \
0       `8830210000008880                                  Money Transfer to    
1       `8830210000008880                                  

In [44]:
# List of columns to process
columns_to_process = ['transaction_id'] # add to account no if there are dual records in it

# Ensure columns are in string format
for col in columns_to_process:
    if col in df.columns:
        df[col] = df[col].astype(str)
        
# Process each column and merge results
processed_dfs = [process_column(df, col) for col in columns_to_process]
# Merge all processed DataFrames
final_df = pd.concat(processed_dfs, ignore_index=True)

# Display the updated DataFrame to verify
df = df.drop_duplicates(keep='first')

# Save the modified DataFrame to a new CSV file if needed
#final_df.to_csv('D:\\test\\excel2test_Cleaned.csv', index=False)
df = final_df
df.describe

<bound method NDFrame.describe of     s_no acknowledgement_no           transaction_id layer  \
0      1    `30209240022092  `PUNBR52024090612314232     2   
1      2    `30209240022092  `PUNBR52024090612314232     2   
2      3    `30209240022092  `PUNBR52024090612314232     2   
3      4    `30209240022092  `PUNBR52024090612314232     2   
4      5    `30209240022092  `PUNBR52024090612314232     2   
..   ...                ...                      ...   ...   
152  149    `30209240022092  `DBSSR52024090605166760     3   
153  150    `30209240022092  `UJVNN52024090646162643     3   
154  151    `30209240022092  `UJVNN52024090646162610     3   
155  152    `30209240022092          `OtrCyfe9snuywn     4   
156  153    `30209240022092            `425011000249     5   

            to_account_no action_taken_by_bank_wallet_pgpa_merchant_insurance  \
0       `8830210000008880                                  Money Transfer to    
1       `8830210000008880                                  

In [45]:
df['amount']=df['amount'].astype(float)
df['layer'] = df['layer'].astype(int)

#df = df.drop(['unique_id'],axis=1)
df = df.drop(['s_no'],axis=1)

In [46]:
# Define the columns to clean
columns_to_clean = [
    'acknowledgement_no', 'transaction_id', 'layer',
    'from_account_no', 'utr_number', 'amount','to_account_no'
]

# Apply the cleaning function to each specified column
for col in columns_to_clean:
    if col in df.columns:
        df[col] = clean_column_data(df[col])

In [47]:
#df['unique_id'] = pd.Series(range(1, len(df) + 1))
columns_to_update = ['to_account_no', 'from_account_no', 'ifsc_code', 'utr_number', 'remarks']
df.isnull().sum() + (df == '').sum()

acknowledgement_no                                       0
transaction_id                                           0
layer                                                    0
to_account_no                                            0
action_taken_by_bank_wallet_pgpa_merchant_insurance      0
bank_wallet_pgpa_merchant__insurance                     0
from_account_no                                          0
ifsc_code                                                0
cheque_no                                                0
mid                                                    152
tid                                                    152
approval_code                                          152
merchant_name                                          152
transaction_date                                         0
utr_number                                               0
amount                                                   0
reference_no                                           1

In [48]:

# Replace empty strings and null values with 'NaaN'
df[columns_to_update] = df[columns_to_update].replace('', 'NaaN').fillna('NaaN')
df.isnull().sum() + (df == '').sum()

acknowledgement_no                                       0
transaction_id                                           0
layer                                                    0
to_account_no                                            0
action_taken_by_bank_wallet_pgpa_merchant_insurance      0
bank_wallet_pgpa_merchant__insurance                     0
from_account_no                                          0
ifsc_code                                                0
cheque_no                                                0
mid                                                    152
tid                                                    152
approval_code                                          152
merchant_name                                          152
transaction_date                                         0
utr_number                                               0
amount                                                   0
reference_no                                           1

In [49]:
df['layer'] = pd.to_numeric(df['layer'], errors='coerce')
print(df['layer'].unique())

[2 3 1 4 5 6 7]


In [50]:
df['to_account_no'] = df['to_account_no'].apply(clean_account_no)
df['from_account_no'].unique()

array(['24610100000643', '110189607716', '1607104000080370',
       '20200057515505', '20200059117762', '2401216960825134',
       '2401227259431241', '20200059779220', '21360100147753',
       '0179053000017033', '60496115000', '60501402002', '110196345628',
       '2401216960825130', '110171398496', '746310110004930',
       '19680100099710', '0525104000462419', '28430110056608',
       '24610100005204', '1146110010053655', '1146110010053650',
       '18670100063718', '079801513015', '20200050694941', '110171463270',
       '2401227259431240', '919376719291', '20200057674862',
       '10111712000', '8830210000008884', '2006104000039817', 'NaaN',
       '43017388358', '43280321955', '110170638899', '20200060776541',
       '20200060503423', '1564019694617845', '110134809783',
       '20200060530420', '1001014000220', '022751400002853',
       '20200058727202', '924010041487963', '918955891544', '76852024',
       '76850584', '76851397', '76851639', '76854670',
       'TERMINAL ID00216

In [51]:
ackno = df['acknowledgement_no'].unique()
ackname = 'Transaction Flow Graph For (Ack No): ' + str(ackno)
ackname

"Transaction Flow Graph For (Ack No): ['30209240022092']"

In [52]:
df.to_csv('D:\\test\\finaltest.csv', index=False)

In [53]:
# Create a directed graph using Graphviz with hierarchical layout
dot = Digraph()

# Set the graph layout to be hierarchical (left-right) and use L-shaped edges
dot.attr(rankdir='LR', splines='ortho')

# Get the distinct unique layers from the dataset
layers = sorted(df['layer'].unique())

# Add a title at the top
dot.attr(label=ackname, fontsize='20', labelloc='t', fontcolor='black')

with dot.subgraph(name='cluster_legend') as legend:
    legend.attr(label="Legend", fontsize='14', style='dashed', rank='source')  # Use rank='source' to push it to the top
    legend.node('withdrawal', label="Withdrawal (Blue)", shape='box', color='#0000ff')
    legend.node('on_hold', label="On Hold (Red)", shape='box', color='#ff0000')
    legend.node('normal', label="Normal Transaction (Black)", shape='box', color='black')
    legend.node('legend', label= f"""<<TABLE BORDER="0" CELLBORDER="0">
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="orange" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Account No. (Orange)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="#036100" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Transaction ID (Green)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="blue" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>IFSC_code (Black)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="red" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Amount (Red)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="#6b3700" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Transaction Date (Brown)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="blue" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Action Taken (Blue)</TD>
        </TR>
        <TR>
            <TD><TABLE BORDER="0" CELLBORDER="0"><TR><TD BGCOLOR="black" WIDTH="20" HEIGHT="20"></TD></TR></TABLE></TD>
            <TD>Remarks (Black)</TD>
        </TR>
    </TABLE>>""", shape='plaintext')  # Ensure this is in the correct format

# Initialize the previous layer nodes (starting with an empty set)
previous_layer_nodes = None

# Set to track already added edges to avoid duplicates
added_edges = set()

# Loop through each distinct layer in the dataset
for idx, layer in enumerate(layers):
    
    # Filter the records for the current layer
    layer_records = df[df['layer'] == layer]
    print(layer,':', layer_records['to_account_no'])
    # Select relevant columns from the filtered dataset
    layer_nodes = layer_records[['from_account_no', 'to_account_no', 'transaction_id', 'transaction_date', 'ifsc_code', 'amount', 'remarks', 'action_taken_by_bank_wallet_pgpa_merchant_insurance']]
    
    # Add nodes for the current layer (from_account_no as nodes, with rectangular shape)
    for _, row in layer_nodes.iterrows():
        # Default color is black
        node_color = 'black'
        
        # Change node color if 'WITHDRAWAL' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#0000ff'  # Blue color for withdrawal
            
        # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
        elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
            node_color = '#ff0000'  # Red color for on hold
        # Create the HTML-like label for each node with colored text
        # Create the HTML-like label for each node with colored text
        label = f"""<<TABLE BORDER="0" CELLBORDER="0">
            <TR><TD><FONT COLOR="orange" POINT-SIZE="12"><B>{row['from_account_no']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="#036100"><B>{row['transaction_id']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="blue"><B>{row['ifsc_code']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="red" POINT-SIZE="15"><B>{format_amount_indian(row['amount'])}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="#6b3700">{row['transaction_date']}</FONT></TD></TR>
            <TR><TD><FONT COLOR="blue">{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}</FONT></TD></TR>
        </TABLE>>"""

        # Add node with the specific color and rectangular shape
        dot.node(
            str(row['from_account_no']), 
            label=label,
            shape='box',
            color=node_color
        )

        # Check for NaN in from_account_no
        if str(row['from_account_no']).upper() == "NAAN":
            # Create a unique identifier for each NaN node based on its transaction details
            nan_node_id = f"NaaN_{row['transaction_id']}"  # Unique ID for this specific NaaN transaction
            node_color = '#cccccc'
            if 'WITHDRAWAL' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                node_color = '#0000ff'  # Blue color for withdrawal
            
            # Change node color if 'ON HOLD' is in action_taken_by_bank_wallet_pgpa_merchant_insurance
            elif 'ON HOLD' in row['action_taken_by_bank_wallet_pgpa_merchant_insurance'].upper():
                node_color = '#ff0000'  # Red color for on hold
            remarks = split_text(row['remarks'], max_width=30) 
            label = f"""<<TABLE BORDER="0" CELLBORDER="0">
            <TR><TD><FONT COLOR="orange" POINT-SIZE="12"><B>{row['from_account_no']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="#036100"><B>{row['transaction_id']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="blue"><B>{row['ifsc_code']}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="red" POINT-SIZE="15"><B>{format_amount_indian(row['amount'])}</B></FONT></TD></TR>
            <TR><TD><FONT COLOR="black">{row['transaction_date']}</FONT></TD></TR>
            <TR><TD><FONT COLOR="#6b3700">{row['action_taken_by_bank_wallet_pgpa_merchant_insurance']}</FONT></TD></TR>
            <TR><TD><FONT COLOR="black">{remarks}</FONT></TD></TR>
        </TABLE>>"""
            dot.node(nan_node_id, label=label, shape='ellipse', color=node_color)

            # Add an edge from the current node to the unique NaaN node
            edge = (str(row['to_account_no']), nan_node_id)
            if edge not in added_edges:
                dot.edge(str(row['to_account_no']), nan_node_id)
                added_edges.add(edge)

    # If this is not the first layer, add edges between the previous layer and the current one
    if previous_layer_nodes is not None:
        # Add edges based on "to_account_no" in the current layer matching "from_account_no" in the previous layer
        for _, row in layer_nodes.iterrows():
            matching_previous_layer_nodes = previous_layer_nodes[previous_layer_nodes['from_account_no'] == row['to_account_no']]
            
            for _, prev_row in matching_previous_layer_nodes.iterrows():
                # Create an identifier for the edge to check for duplicates
                edge = (str(prev_row['from_account_no']), str(row['from_account_no']))
                
                # Add edge if it hasn't been added yet
                if edge not in added_edges:
                    dot.edge(str(prev_row['from_account_no']), str(row['from_account_no']))
                    added_edges.add(edge)  # Track the added edge

    # Set the current layer as the previous layer for the next iteration
    previous_layer_nodes = layer_nodes

# Save and view the graph (optional)
dot.render('dfs_transaction_graph_with_individual_nan_colors', format='png', cleanup=True)
dot.view()

1 : 23    8830210000008880
34      10832030036580
38    8830210000008880
39         10047593897
40         10047593897
41         10047593897
42         43017388358
43         43280321955
44         43017388358
45         43280321955
52    8830210000008880
Name: to_account_no, dtype: object
2 : 0      8830210000008880
1      8830210000008880
2      8830210000008880
3      8830210000008880
4      8830210000008880
5      8830210000008880
6      8830210000008880
7      8830210000008880
8      8830210000008880
9      8830210000008880
10     8830210000008880
11     8830210000008880
12     8830210000008880
13     2401216960825130
21     8830210000008880
22     8830210000008880
24     8830210000008880
25     8830210000008880
26     8830210000008880
30     2401227259431240
46          43017388358
47          43017388358
48          43017388358
49          43017388358
50          43017388358
51          43017388358
53          43280321955
54          43280321955
55          43280321955
56      

'dfs_transaction_graph_with_individual_nan_colors.pdf'